In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response = llm.invoke('First person to land on the moon is')
print(response.content)

The first person to land on the moon is Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.intuit.com/job/mountain-view/senior-machine-learning-engineer/27595/67584489104")
page_data = loader.load().pop().page_content

In [5]:
from langchain_core.prompts import PromptTemplate

In [14]:
prompt_extract = PromptTemplate.from_template(
    """" 
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job postings anf return them in JSON format containing 
    the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "role": "Senior Machine Learning Engineer",
  "experience": "6+ years",
  "skills": [
    "Data Science tools and frameworks (i.e. Python, Scikit, NLTK, Numpy, Pandas, TensorFlow, Keras, R, Spark)",
    "Machine learning techniques (i.e. classification, regression, and clustering)",
    "Machine learning principles (training, validation, etc.)",
    "Data query and data processing tools (i.e. SQL)",
    "Computer science fundamentals: data structures, algorithms, performance complexity, and implications of computer architecture on software performance",
    "Software engineering fundamentals: version control systems (i.e. Git, Github) and workflows",
    "GPU acceleration (i.e. CUDA and cuDNN)",
    "Cloud technologies (i.e. AWS and GCP)"
  ],
  "description": "In this role, you’ll be embedded inside a vibrant team of data scientists. You’ll be expected to help conceive, code, and deploy data science models at scale using the latest industry tools. Important skills include 

In [7]:
type(res.content)

str

In [15]:
from langchain_core.output_parsers import JsonOutputParser

jp = JsonOutputParser()
json_res = jp.parse(res.content)
json_res

{'role': 'Senior Machine Learning Engineer',
 'experience': '6+ years',
 'skills': ['Data Science tools and frameworks (i.e. Python, Scikit, NLTK, Numpy, Pandas, TensorFlow, Keras, R, Spark)',
  'Machine learning techniques (i.e. classification, regression, and clustering)',
  'Machine learning principles (training, validation, etc.)',
  'Data query and data processing tools (i.e. SQL)',
  'Computer science fundamentals: data structures, algorithms, performance complexity, and implications of computer architecture on software performance',
  'Software engineering fundamentals: version control systems (i.e. Git, Github) and workflows',
  'GPU acceleration (i.e. CUDA and cuDNN)',
  'Cloud technologies (i.e. AWS and GCP)'],
 'description': 'In this role, you’ll be embedded inside a vibrant team of data scientists. You’ll be expected to help conceive, code, and deploy data science models at scale using the latest industry tools. Important skills include data wrangling, feature engineering,

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

data = pd.read_csv('my_portfolio.csv')
data

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in data.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=['Experience in Python'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [16]:
job = json_res
job['skills']

['Data Science tools and frameworks (i.e. Python, Scikit, NLTK, Numpy, Pandas, TensorFlow, Keras, R, Spark)',
 'Machine learning techniques (i.e. classification, regression, and clustering)',
 'Machine learning principles (training, validation, etc.)',
 'Data query and data processing tools (i.e. SQL)',
 'Computer science fundamentals: data structures, algorithms, performance complexity, and implications of computer architecture on software performance',
 'Software engineering fundamentals: version control systems (i.e. Git, Github) and workflows',
 'GPU acceleration (i.e. CUDA and cuDNN)',
 'Cloud technologies (i.e. AWS and GCP)']

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Abhisumat, a business development executive at Deloitte. Deloitte is a Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Deloitte 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Deloitte's portfolio: {link_list}
        Remember you are Abhisumat, BDE at Deloitte. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Solutions for Your Business

Dear Hiring Manager,

I came across the job description for a Senior Machine Learning Engineer at your organization and was impressed by the scope of the role. As a Business Development Executive at Deloitte, I'd like to introduce you to our team of experts who can help conceive, code, and deploy data science models at scale using the latest industry tools.

At Deloitte, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of data scientists and machine learning engineers have extensive experience in developing and deploying machine learning models using a range of tools and technologies, including Python, Scikit, NLTK, Numpy, Pandas, TensorFlow, Keras, and R.

We've successfully delivered numerous projects that showcase our expertise in machine learning and data science. Some notable examples